# A Hitchhiker's Guide to RNA Velocity

This guide servers as a "how to" for using RNA velocity and associated python packages.

Created by Richard Perez

Package Requirements:

Anaconda package - To handles package dependencies.

Velocyto package - Preliminary processing of single-cell RNA sequencing data (scRNAseq).

Scvelo package   - Really excellent interface for post-processing of scRNAseq data.

Loompy package   - To handle and manage loom files.

Scanpy package   - My go to scRNAseq package for python and works well with velopy.


## Preprocessing: from .bam to .loom

At this point you need to have your scRNAseq data processed to the point where it is in a .bam file format. 

For this tutorial, I will be using a reference .h5ad object which has already been filtered for doublets by two independent methods (Demuxlet and Doublet Detection). Additionally, this object contains pertinent covariates I would like to transfer into my RNA velocity .h5ad object in the end.

### Step 1: Determine which barcodes you want to process

I am using barcodes of the cells from my reference .h5ad object. They should be .tsv format. This script opens the object and collects pertinent barcodes based on batch.

In [ ]:
import pandas as pd
import logging
import numpy as np
import scanpy.api as sc

# Configure logging
logging.basicConfig(level=logging.INFO)

# Where is the batch specific covariate .csv?
filepath = 'SLEcrossX_processed.h5ad'
adata = sc.read(filepath)
adata.obs['well'] = adata.obs['well'].astype('object')
logging.info(str('data ' + str(adata)))
master_batch_list = np.unique(adata.obs['well'].tolist())
logging.info(str('well ' + str(adata.obs['well'])))
logging.info(str('all unique wells ' + str(master_batch_list)))

def getbarcodes(adata, batch):
    barcodes = adata[adata.obs['well'].isin([batch])].obs['well'].index.tolist()
    logging.info(str('Barcodes found for batch ' + batch))
    logging.info(str('Number of Cells ' + str(len(barcodes))))
    savepath = str('/tsv/' + batch + '.tsv')
    with open(savepath, 'w') as file:
        for code in barcodes:
            file.write("%s \n" % code)

for batch in master_batch_list:
    getbarcodes(adata, batch)


### Step 2: Run Velocyto

You will need to submit this as a job and it will likely take a few hours.

-b path to barcodes we just collected.

-m mask reccomended when preprocessing (see velocyto documentation).

-o name output directory

-e name of output file (.loom)


In [ ]:
velocyto run -b '/tsv/YE_8-16-1.tsv' -m '/hg19_rmsk.gtf' -o '/loom' -e 'YE-8-16-1' '/YE_8-16-1/outs/possorted_genome_bam.bam' '/10x.ref/refdata-cellranger-hg19-1.2.0/genes/genes.gtf'

### Step 3: Combine all loom files

For our dataset, it was composed about about a dozen different batches and each needed to be processed separately. We now need to combine all of them.

In [ ]:
import loompy
import os
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
# Initialize files list
files = []
# Get current working directory
cwd = os.getcwd()
# Name output file automatically
output_filename = str('crossx.loom')
# Scan files in working directory for files ending with .loom and append them to the files list.
for file in os.listdir(cwd):
    logging.info(str(file))
    if file.endswith(".loom"):
        if os.path.join(cwd, file) != output_filename:
            files.append(os.path.join(cwd, file))


logging.info(str(files))
# Combine loom files
loompy.combine(files, output_filename, key="Accession")

### Step 4: Transfer covariates from reference .h5ad

Barcodes in crossx.loom will look like "batch:ATGATGATGATGATGAx". The original barcode was "ATGATGATGATGATGA-4-000001". Notice how the preprocessing trimmed the barcode? This is not good so we need to ensure that the covariates we are transfering over are associated with the correct barcode. This is an inefficient script but it works.

In [ ]:
import numpy as np
import scanpy.api as sc
import logging
import scvelo as scv
import pandas as pd

##################
# Configure file #
##################
sc.settings.verbosity = 2
sc.settings.autoshow = False
logging.basicConfig(level=logging.INFO)

adata = scv.read(loompath, cache=True)
logging.info(str('loom data structure details: ' + str(adata)))
rnavelobarcodes = adata.obs_names.tolist()
batch = []
BARCODES = []
for barcode in rnavelobarcodes:
    tmp = barcode.split(':')[0]
    # This one batch does not follow the conventions of the others.
    if tmp[:5] != 'YE110':
        # loom file names are a little different than the batch names.
        tmp = tmp.replace('-', '_', 1)
    batch.append(tmp)
    newbarcode = barcode.split(':')[1]
    newbarcode = newbarcode[:-1]
    BARCODES.append(newbarcode)
adata.obs['barcodes'] = BARCODES
adata.obs['batchcore'] = batch
logging.info(str('Updated data structure details: ' + str(adata)))

refadata = sc.read(refh5adpath, cache=True)
logging.info(str('Reference data structure details: ' + str(refadata)))

# Covariates I care about.
DZ = []
IND = []
WELL = []
CT = []
LOUVAIN = []
for ii in range(len(BARCODES)):
    refbarcodes = refadata.obs['well'][refadata.obs['well']==batch[ii]].index.tolist()

    for jj in range(len(refbarcodes)):
        refbarcodes[jj] = refbarcodes[jj].split('-')[0]

    dz = refadata.obs['disease_cov'][refadata.obs['well']==batch[ii]].values.tolist()
    ind = refadata.obs['ind_cov'][refadata.obs['well']==batch[ii]].values.tolist()
    well = refadata.obs['well'][refadata.obs['well']==batch[ii]].values.tolist()
    ct = refadata.obs['ct_cov'][refadata.obs['well']==batch[ii]].values.tolist()
    louvain = refadata.obs['louvain'][refadata.obs['well']==batch[ii]].values.tolist()

    DZ.append(dz[refbarcodes.index(BARCODES[ii])])
    IND.append(ind[refbarcodes.index(BARCODES[ii])])
    WELL.append(well[refbarcodes.index(BARCODES[ii])])
    CT.append(ct[refbarcodes.index(BARCODES[ii])])
    LOUVAIN.append(louvain[refbarcodes.index(BARCODES[ii])])

adata.obs['disease_cov'] = DZ
adata.obs['ind_cov'] = IND
adata.obs['ct_cov'] = CT
adata.obs['well'] = WELL
adata.obs['louvain'] = LOUVAIN
logging.info(str('Updated data structure details: ' + str(adata)))
logging.info(str('Saving...'))
adata.write(savepath)

### Step 5: Process data

Now that our data is in a h5ad object format, we can process it with scanpy. Below are basic codes to do that. 

Note: I already filtered out doublets and that script is not shown below. Also note that since I am pulling barcodes from a reference h5ad object which was previously processed, I did not remove additional cells when processing the new h5ad object. However that code will be included below. I also did not run cell identity or louvain clustering again for our new h5ad object. I pulled those as covariates from the reference and they serve as a sanity check for barcode matching.

#### Step 5a:

In [ ]:
import numpy as np
import scanpy.api as sc
import logging

##################
# Configure file #
##################
sc.settings.verbosity = 2
sc.settings.autoshow = False
logging.basicConfig(level=logging.INFO)

####################
# Basic processing #
####################
adata = sc.read(filepath)
adata.obs['batchcore'] = adata.obs['batchcore'].astype('category')
adata.var_names_make_unique()
logging.info(str('Data structure details: ' + str(adata)))
# Extract list of genes
genelist = adata.var_names.tolist()
# Find mitochondrial genes
mito_genes_names = [gn for gn in genelist if gn.startswith('MT-')]
logging.info(str('Mito genes: ' + str(mito_genes_names)))
# Find indices of mitochondrial genes
mito_genes = [genelist.index(gn) for gn in mito_genes_names]
# For each cell compute fraction of counts in mito genes vs. all genes
adata.obs['percent_mito'] = np.ravel(np.sum(adata[:, mito_genes].X, axis=1)) / np.ravel(np.sum(adata.X, axis=1))
# Add the total counts per cell as observations-annotation to adata
adata.obs['n_counts'] = np.ravel(adata.X.sum(axis=1))
logging.info('Filtering cells')
# Filter cells that have more than 2500 genes or more than 10% of counts coming from mitochondrial genes.
# These are likely outliers.
adata = adata[adata.obs['percent_mito'] < 0.10]
logging.info(str('Data structure details: ' + str(adata)))
# Filter cells with abnormally low gene counts, high gene counts.
sc.pp.filter_cells(adata, min_genes=100)
logging.info(str('Data structure details: ' + str(adata)))
sc.pp.filter_cells(adata, max_genes=2500)
logging.info(str('Data structure details: ' + str(adata)))
logging.info('Saving raw and raw counts')
adata.uns['raw_counts'] = adata.X
adata.raw = sc.pp.log1p(adata, copy=True)
logging.info('Normalizing total counts to 10,000')
sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
logging.info('Filtering genes')
filter_result = sc.pp.filter_genes_dispersion(adata.X, log=True)
adata = adata[:, filter_result.gene_subset]
logging.info('Log transforming data')
sc.pp.log1p(adata)
logging.info(str('Data structure details: ' + str(adata)))
logging.info('Regressing out variance within total nUMIs and % mitochondrial UMIs')
sc.pp.regress_out(adata, ['n_counts', 'percent_mito'])
logging.info('Batch correcting by regressing out batch variance')
sc.pp.regress_out(adata, ['batchcore'])
logging.info('Scaling expression data')
sc.pp.scale(adata, max_value=10)
logging.info(str('Data structure details: ' + str(adata)))
# Unique list of individuals
people = np.unique(adata.obs['ind_cov'].values.tolist())
# Allocate space for total PMBCs per individual.
total_pbmcs = dict.fromkeys(people)
for p in people:
    total_pbmcs[p] = len(adata.obs_names[adata.obs['ind_cov'] == p])
adata.uns['total_pbmcs'] = total_pbmcs
logging.info('Saving processed data')
adata.write(savepath)
logging.info('Basic processing complete.')

#### Step 5b:

In [ ]:
import scanpy.api as sc
import logging

##################
# Configure file #
##################
sc.settings.verbosity = 2
sc.settings.autoshow = False
logging.basicConfig(level=logging.INFO)
adata = sc.read(filepath)

#######################
# Louvain and friends #
#######################
# Set parameters
intialization = 1
n_components = 20
resolution = 1
# Run louvain clustering on theoretical future gene expression per cell
logging.info('Estimating louvain cluster identities for gene expression values.')
sc.pp.pca(adata, random_state=intialization)
logging.info('PCA complete.')
sc.pp.neighbors(adata)
logging.info('KNN complete.')
sc.tl.diffmap(adata)
logging.info('diffmap complete.')
sc.tl.louvain(adata, random_state=intialization, resolution=resolution)
logging.info('Louvain complete.')
sc.tl.paga(adata)
sc.pl.paga(adata, random_state=intialization, show=False)
logging.info('paga complete.')
sc.tl.umap(adata, random_state=intialization, init_pos='paga')
logging.info('UMAP complete.')
sc.tl.tsne(adata, n_pcs=10)
logging.info('TSNE complete.')
# First PC for ordering of cells in the umap
adata.obs['ordering_UMAP'] = sc.pp.pca(adata.obsm['X_umap'], n_comps=1, copy=True)
adata.write(filepath)
logging.info('Basic analysis complete.')

#### Step 5c

In [ ]:
import numpy as np
import scanpy.api as sc
import logging

##################
# Configure file #
##################
sc.settings.verbosity = 2
sc.settings.autoshow = False
logging.basicConfig(level=logging.INFO)
adata = sc.read(filepath)

annotation = {'CD8+ T': ['CD8A', 'CD8B', 'CD3D'], 'CD4+ T': ['CD3D', 'ANK3', 'IL7R'],
              'CD14 Mono': ['CD14', 'LYZ', 'CCL2', 'S100A9'], 'CD16 Mono': ['FCGR3A', 'MS4A7', 'VMO1'],
              'B': ['CD19', 'MS4A1', 'CD79A'], 'NK': ['KLRF1', 'GNLY', 'NKG7'],
              'cDC': ['HLA.DQA1', 'FCER1A', 'GPR183'], 'pDC': ['CLEC4C', 'TSPAN13', 'IGJ'],
              'MK': ['PPBP', 'GNG11']}

######################
# Identify cell type #
######################
# Differentially expressed genes
Genes = []
for ct in annotation.keys():
    Genes.append(annotation[ct])
Genes = [item for sublist in Genes for item in sublist]
# Determine cell types
ct_cov = np.asarray(adata.obs['disease_cov'].values.tolist())
for ii in np.unique(adata.obs['louvain'].tolist()):
    currntsum = -1
    for ct in annotation.keys():
        ctsum = []
        for Gn in Genes:
            if Gn in annotation[ct]:
                ctsum.append(np.nanmean(adata.X[:, adata.var_names.isin([Gn])][adata.obs['louvain'] == ii]))
            else:
                ctsum.append(
                    np.nanmean(adata.X[:, adata.var_names.isin([Gn])][adata.obs['louvain'] == ii]) * -1)
        ctsum = np.nanmean(np.asarray(ctsum))
        # Assign cell type
        if ctsum > currntsum:
            ct_cov[adata.obs['louvain'] == ii] = ct
            currntsum = ctsum
        else:
            continue
adata.obs['ct_cov'] = ct_cov
adata.obs['ct_cov'] = adata.obs['ct_cov'].astype('category')
adata.write(filepath)

## Step 6: Run RNA Velocity

In [ ]:
import numpy as np
import scanpy.api as sc
import logging
import pandas as pd

####################
#  Configure file  #
####################
# Configure logging
logging.basicConfig(level=logging.INFO)
# Configure scanpy settings
sc.settings.verbosity = 2
sc.settings.autoshow = False
sc.settings.set_figure_params(dpi_save=1000, format='png')

filepath = 'crossx_rnavelo_processed.h5ad'
adata = scv.read(filepath, cache=True)
# Only run for Monocytes
adata = adata[adata.obs['ct_cov'].isin(['CD14 Mono', 'CD16 Mono'])]
logging.info(str('data structure details: ' + str(adata)))
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.tl.louvain(adata, resolution=1, random_state=1)
sns.set(font_scale=1.25)
sc.settings.set_figure_params(dpi_save=1000, format='png', transparent=True)
color_map = [
    '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
    '#9467bd', '#8c564b', '#e377c2',
    '#bcbd22', '#17becf',
    '#aec7e8', '#ffbb78', '#98df8a', '#ff9896',
    '#c5b0d5', '#c49c94', '#f7b6d2',
    '#dbdb8d', '#9edae5',
    '#ad494a', '#8c6d31']

sc.pl.umap(adata, color='louvain', show=False, save='louvain.png')
sc.pl.umap(adata, color='disease_cov', show=False, save='disease.png')
sc.pl.umap(adata, color='well', show=False, save='well.png')
sc.pl.umap(adata, color='batchcore', show=False, save='batch.png')
sc.pl.umap(adata, color='ct_cov', show=False, save='ct_cov.png')
sc.pl.umap(adata, color=['IFI27'], show=False, save='IFI27.png')

def run_rnavelo(adata, dz_typ):
    adata = adata[adata.obs['disease_cov']==dz_typ]
    # I subsample SLE for compute time purposes.
    if dz_typ == 'sle':
        sc.pp.subsample(adata, fraction=0.25)
    logging.info(str('Data structure details: ' + str(adata)))
    scv.pp.show_proportions(adata)
    scv.pp.moments(adata)
    logging.info(str('Data structure details: ' + str(adata)))
    scv.tl.velocity(adata)
    scv.tl.velocity_graph(adata, n_recurse_neighbors=2)
    scv.tl.velocity_embedding(adata, basis='umap')
    scv.tl.terminal_states(adata)
    scv.pl.velocity_embedding_grid(adata, basis='umap', show=False, save=str([str(dz_typ)+'_rnavelocity_grid.png']), dpi=1000, frameon=False)
    scv.pl.velocity_embedding(adata, basis='umap', show=False, save=str([str(dz_typ)+'_rnavelocity.png']), dpi=1000, frameon=False)
    scv.pl.scatter(adata, basis='umap', color=['root', 'end'], perc=[2,98], show=False, save=str([str(dz_typ)+'_markov.png']), dpi=1000)

run_rnavelo(adata, 'sle')
run_rnavelo(adata, 'healthy')


## Tutorial complete!